In [0]:
dbutils.widgets.text("proc_date", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../../../utils/common

In [0]:
# dbutils.widgets.text("proc_date", "")
# proc_date = dbutils.widgets.get("proc_date")
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
create_temp_view_with_clean_columns(
    spark,
    catalog_name,
    'udp_wcm_bronze_rop',
    'store_profile_new',
    proc_date,
    'temp_rop_store_profile_new'
)

In [0]:
spark.sql(f"""CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_silver_rop.store_profile_new LIKE {catalog_name}.udp_wcm_bronze_rop.store_profile_new""")

In [0]:
spark.sql(f"""ALTER TABLE {catalog_name}.udp_wcm_silver_rop.store_profile_new
SET TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
""")

In [0]:
spark.sql(f"""
    DELETE FROM {catalog_name}.udp_wcm_silver_rop.store_profile_new
    WHERE PROC_DATE = DATE('{proc_date}')
""")

In [0]:
spark.sql(f"""
        INSERT INTO {catalog_name}.udp_wcm_silver_rop.store_profile_new 
        SELECT no,
            store_id,
            store_name,
            business_unit,
            sap_pc,
            region,
            city,
            district,
            ward,
            address,
            longtitude,
            latitude,
            open_date,
            ground,
            front_m,
            area_m2,
            biz_area_m2,
            size_v,
            assortment_v,
            concept,
            warehouse_size,
            lease_term,
            status,
            proc_date,
            concept_id,
            store_type,
            file_creation_ts
        FROM temp_rop_store_profile_new
        WHERE proc_date = DATE('{proc_date}')
    """)